In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [172]:
import itertools
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

#from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout,regularizers
from keras.preprocessing import text, sequence
from keras import utils

# This code was tested with TensorFlow v1.4
print("You have TensorFlow version", tf.__version__)

You have TensorFlow version 1.3.0


In [173]:
data = pd.read_table("DataSet/QnA/train_1000.label",encoding="ISO-8859-1",header=-1)

In [174]:
def dataProcessor(text):
    text1 = text.split(" ")
    return ([text1[0]," ".join(text1[1:])])

def dataAppenderMultipleFiles(dataList):
    list1 = []
    list2 = []
    #llist = []
    for i in data[0]:
        l1,l2 = dataProcessor(i)
        #print(l1)
        #print(l2)
        list1+=[l1]
        list2+=[l2]
    #print(list1)
    return (list1,list2)
    

In [175]:
#dataPrep = data[0].apply(lambda x: dataProcessor(x))

list1 = []
list2 = []
llist = []
for i in data[0]:
    l1,l2 = dataProcessor(i)
    #print(l1)
    list1+=[l1]
    list2+=[l2]
#print(list1, list)    
llist+= [list1,list2]
#print(llist)

In [176]:
#dataPrep.reshape(2,len(dataPrep))

llist = [[],[]]
for i in os.listdir("DataSet/QnA/"):
    #print(i)
    if i.find("train") != -1:
        fullPath = "".join(["DataSet/QnA/",i])
        #print(fullPath)
        data = pd.read_table("".join(["DataSet/QnA/",i]),encoding="ISO-8859-1",header=-1)
        l1,l2= dataAppenderMultipleFiles(data[0])
        llist[0]+=l1
        llist[1]+=l2

In [177]:
len(llist)
#a = [[1,2],[3,4],[5,6]]

2

In [257]:
'''
newData = pd.DataFrame(columns=["target","Question"])
newData["target"] = llist[0]
newData["Question"] = llist[1]
newData.append(llist)
'''
newData = pd.DataFrame(llist[0],columns=["target"])
newData["Question"] = llist[1]

In [258]:
newData['target'].value_counts()

HUM:ind           2691
LOC:other         1307
DESC:def          1221
NUM:count          985
DESC:desc          774
DESC:manner        766
NUM:date           650
ENTY:other         628
ENTY:cremat        595
DESC:reason        543
HUM:gr             529
LOC:country        425
LOC:city           376
ENTY:animal        365
ENTY:dismed        291
ENTY:termeq        271
ENTY:food          266
LOC:state          201
NUM:period         197
ABBR:exp           195
NUM:money          183
ENTY:event         173
ENTY:sport         165
NUM:other          154
HUM:desc           137
ENTY:product       130
ENTY:substance     124
ENTY:color         119
ENTY:techmeth      111
NUM:dist            86
NUM:perc            82
ENTY:word           71
ENTY:veh            68
LOC:mount           67
HUM:title           67
ENTY:body           54
ABBR:abb            46
ENTY:lang           45
ENTY:plant          38
ENTY:instru         37
NUM:speed           36
NUM:volsize         32
ENTY:symbol         31
ENTY:letter

In [259]:
# Split data into train and test
train_size = int(len(newData) * .8)
print("Train size: %d" % train_size)
print("Test size: %d" % (len(newData) - train_size))

Train size: 12361
Test size: 3091


In [260]:
train_posts = newData['Question'][:train_size]
train_tags = newData['target'][:train_size]

test_posts = newData['Question'][train_size:]
test_tags = newData['target'][train_size:]

In [261]:
dataPrep[1]

[['ENTY:cremat'], ['What films featured the character Popeye Doyle ?']]

In [262]:
max_words = 50000
#tokenize = text.Tokenizer(num_words=max_words, char_level=False)
tokenize = text.Tokenizer(char_level=False)

In [263]:
tokenize.fit_on_texts(train_posts) # only fit on train
x_train = tokenize.texts_to_matrix(train_posts)
x_test = tokenize.texts_to_matrix(test_posts)

In [264]:
# Use sklearn utility to convert label strings to numbered index
encoder = LabelEncoder()
encoder.fit(train_tags)
y_train = encoder.transform(train_tags)
y_test = encoder.transform(test_tags)

In [265]:
# Converts the labels to a one-hot representation
num_classes = np.max(y_train) + 1
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)
max_words = x_train.shape[1]

In [266]:
#y_train[0]

In [267]:
# Inspect the dimenstions of our training and test data (this is helpful to debug)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

x_train shape: (12361, 8372)
x_test shape: (3091, 8372)
y_train shape: (12361, 50)
y_test shape: (3091, 50)


In [268]:
# This model trains very quickly and 2 epochs are already more than enough
# Training for more epochs will likely lead to overfitting on this dataset
# You can try tweaking these hyperparamaters when using this model with your own data
batch_size = 32
epochs = 30

In [269]:
# Build the model
model = Sequential()
model.add(Dense(512, input_shape=(max_words,),kernel_regularizer=regularizers.l2(0.001),
                activity_regularizer=regularizers.l1(0.001)))
model.add(Activation('relu'))
#model.add(Dropout(0.2))
model.add(Dense(256, input_shape=(max_words,),kernel_regularizer=regularizers.l2(0.001),
                activity_regularizer=regularizers.l1(0.001)))
model.add(Activation('relu'))
#model.add(Dropout(0.2))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [270]:
# model.fit trains the model
# The validation_split param tells Keras what % of our training data should be used in the validation set
# You can see the validation loss decreasing slowly when you run this
# Because val_loss is no longer decreasing we stop training to prevent overfitting
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Train on 11124 samples, validate on 1237 samples
Epoch 1/30
11124/11124 [==============================] - 8s - loss: 3.0664 - acc: 0.5876 - val_loss: 2.5699 - val_acc: 0.6863
Epoch 2/30
11124/11124 [==============================] - 8s - loss: 1.8252 - acc: 0.8512 - val_loss: 2.2070 - val_acc: 0.7251
Epoch 3/30
11124/11124 [==============================] - 7s - loss: 1.4369 - acc: 0.9102 - val_loss: 2.0401 - val_acc: 0.7405
Epoch 4/30
11124/11124 [==============================] - 8s - loss: 1.2112 - acc: 0.9401 - val_loss: 1.9123 - val_acc: 0.7462
Epoch 5/30
11124/11124 [==============================] - 8s - loss: 1.0553 - acc: 0.9640 - val_loss: 1.8347 - val_acc: 0.7599
Epoch 6/30
11124/11124 [==============================] - 7s - loss: 0.9458 - acc: 0.9782 - val_loss: 1.7617 - val_acc: 0.7502
Epoch 7/30
11124/11124 [==============================] - 8s - loss: 0.8679 - acc: 0.9872 - val_loss: 1.6904 - val_acc: 0.7542
Epoch 8/30
11124/11124 [==============================] - 8s -

In [271]:
# Evaluate the accuracy of our trained model
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])

3091/3091 [==============================] - 0s     
Test score: 0.5001113099
Test accuracy: 0.989323843416


In [272]:
model.predict_classes(x_test)

2752/3091 [=========================>....] - ETA: 0s

array([ 9,  9,  5, ...,  3, 34, 30])

In [273]:
y_test

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])